Code in this NB is taken from here: https://gorayni.github.io/blog/2014/06/05/boltzmann-machine.html

In [23]:
from __future__ import division
from enum import Enum
import numpy as np

from tqdm import tqdm

In [38]:
Clamp = Enum('Clamp', 'VISIBLE_UNITS NONE INPUT_UNITS')

class Step:
    def __init__(self, temperature, epochs):
        self.temperature = temperature
        self.epochs = epochs

numInputUnits = 4
numOutputUnits = 4
numHiddenUnits = 2

numVisibleUnits = numInputUnits + numOutputUnits
numUnits = numVisibleUnits+numHiddenUnits

annealingSchedule = [Step(20.,2),
                     Step(15.,2),
                     Step(12.,2),
                     Step(10.,4)]

coocurranceCycle = Step(10.,10)

weights = np.zeros((numUnits,numUnits))
states = np.zeros(numUnits)
energy = np.zeros(numUnits)

connections = np.zeros((numUnits,numUnits), dtype=int)
for i in range(numInputUnits):
    for j in range(i+1,numInputUnits):
        connections[i,j] = 1
    for j in range(1,numHiddenUnits+1):
        connections[i,-j] = 1   
            
for i in range(numOutputUnits):
    for j in range(i+1,numOutputUnits):
        connections[i+numInputUnits,j+numInputUnits] = 1
    for j in range(1,numHiddenUnits+1):
        connections[i+numInputUnits,-j] = 1  

for i in range(numHiddenUnits,0,-1):
    for j in range(i-1,0,-1):
        connections[-i,-j] = 1
        
valid = np.nonzero(connections)
numConnections = np.size(valid[0])
connections[valid] = np.arange(1,numConnections+1)
connections = connections + connections.T - 1


def propagate(temperature, clamp):
    global energy, states, weights
    
    if clamp == Clamp.VISIBLE_UNITS:
        numUnitsToSelect = numHiddenUnits
    elif clamp == Clamp.NONE:
        numUnitsToSelect = numUnits
    else:
        numUnitsToSelect = numHiddenUnits + numOutputUnits

    for i in range(numUnitsToSelect):
        # Calculating the energy of a randomly selected unit    
        unit=numUnits-np.random.randint(1,numUnitsToSelect+1)
        energy[unit] = np.dot(weights[unit,:], states)
        
        p = 1. / (1.+ np.exp(-energy[unit] / temperature))
        states[unit] = 1. if  np.random.uniform() <= p else 0 
                    
def anneal(annealingSchedule, clamp):
    for step in annealingSchedule:
        for epoch in range(step.epochs):
            propagate(step.temperature, clamp)
    
def sumCoocurrance(clamp):                        
    sums = np.zeros(numConnections)
    for epoch in range(coocurranceCycle.epochs):
        propagate(coocurranceCycle.temperature, clamp)
        for i in range(numUnits):
            if(states[i] == 1):
                for j in range(i+1,numUnits):
                    if(connections[i,j]>-1 and states[j] ==1):
                        sums[connections[i,j]] += 1   
    return sums
     
def updateWeights(pplus, pminus):
    global weights
    for i in range(numUnits):
        for j in range(i+1,numUnits):            
            if connections[i,j] > -1:
                index = connections[i,j]
                weights[i,j] += 2*np.sign(pplus[index] - pminus[index])
                weights[j,i] = weights[i,j]

def recall(pattern):
    global states
        
    # Setting pattern to recall
    states[0:numInputUnits] = pattern
     
    # Assigning random values to the hidden and output states
    states[-(numHiddenUnits+numOutputUnits):] = np.random.choice([0,1],numHiddenUnits+numOutputUnits)

    anneal(annealingSchedule, Clamp.INPUT_UNITS)
    
    return states[numInputUnits:numInputUnits+numOutputUnits]
    
def addNoise(pattern):
    probabilities = 0.8*pattern+0.05
    uniform = np.random.random(numVisibleUnits)    
    return (uniform < probabilities).astype(int)
    
    
def learn(patterns):
    global states, weights

    numPatterns = patterns.shape[0]    
    trials=numPatterns*coocurranceCycle.epochs
    # weights = np.zeros((m,m))
            
    for i in tqdm(range(1800)):
        # Positive phase
        pplus = np.zeros(numConnections)
        for pattern in patterns:
            
            # Setting visible units values (inputs and outputs)
            states[0:numVisibleUnits] = addNoise(pattern)
    
            # Assigning random values to the hidden units
            states[-numHiddenUnits:] = np.random.choice([0,1],numHiddenUnits)
    
            anneal(annealingSchedule, Clamp.VISIBLE_UNITS)
            pplus += sumCoocurrance(Clamp.VISIBLE_UNITS)
        pplus /= trials
        
        # Negative phase
        states = np.random.choice([0,1],numUnits)          
        anneal(annealingSchedule, Clamp.NONE)
        pminus = sumCoocurrance(Clamp.NONE) / coocurranceCycle.epochs
       
        updateWeights(pplus,pminus)


patterns = np.array([[1, 0, 0, 0, 1, 0, 0, 0],
                     [0, 1, 0, 0, 0, 1, 0, 0],
                     [0, 0, 1, 0, 0, 0, 1, 0],
                     [0, 0, 0, 1, 0, 0, 0, 1]])
learn(patterns)
print(weights)




100%|██████████| 1800/1800 [00:07<00:00, 231.66it/s]

[[   0.  -16. -418. -420.    0.    0.    0.    0.  670. -274.]
 [ -16.    0. -420. -422.    0.    0.    0.    0. -276.  674.]
 [-418. -420.    0. -802.    0.    0.    0.    0.  388.  404.]
 [-420. -422. -802.    0.    0.    0.    0.    0.  396.  398.]
 [   0.    0.    0.    0.    0.  -18. -410. -410.  668. -296.]
 [   0.    0.    0.    0.  -18.    0. -402. -402. -324.  702.]
 [   0.    0.    0.    0. -410. -402.    0. -784.  388.  380.]
 [   0.    0.    0.    0. -410. -402. -784.    0.  386.  388.]
 [ 670. -276.  388.  396.  668. -324.  388.  386.    0. -312.]
 [-274.  674.  404.  398. -296.  702.  380.  388. -312.    0.]]


In [20]:
weights.shape

(10, 10)

In [67]:
for _ in range(1000):
    res = []
    for pattern in patterns:
        # print(pattern[:4], recall(np.array(pattern[:4])))
        res.append(np.array_equal(pattern[:4], recall(np.array(pattern[:4]))))

    print(sum(res))

3
3
2
2
3
3
0
2
2
2
4
2
1
3
3
3
2
3
3
2
2
1
3
1
1
2
2
2
3
4
3
2
1
2
1
3
2
3
2
3
4
2
2
2
2
4
1
2
1
2
1
2
2
3
2
2
4
4
2
3
2
2
3
2
2
3
3
2
2
4
2
2
3
2
2
2
4
3
3
2
2
2
4
2
2
1
3
2
1
2
3
2
3
3
3
2
3
1
3
3
2
3
3
2
2
3
3
2
2
0
3
1
3
4
1
2
2
1
2
3
4
2
1
2
2
2
2
4
2
0
0
2
2
0
2
2
2
3
2
2
3
4
3
2
2
3
3
3
3
3
3
2
4
2
1
1
2
0
2
2
2
2
2
2
3
3
1
4
3
2
3
2
2
2
4
1
3
2
2
3
2
1
3
1
2
1
1
3
2
2
3
1
2
3
2
1
3
2
2
0
2
3
3
2
3
3
3
1
3
2
2
2
2
2
3
2
2
0
2
2
4
1
2
3
2
2
2
1
1
3
2
3
2
2
2
3
3
2
2
0
2
1
2
2
4
2
2
2
3
2
2
3
2
3
1
1
2
3
2
2
3
3
2
3
4
2
2
2
3
2
2
3
3
2
2
2
3
0
2
2
2
3
2
3
2
3
3
3
3
1
4
2
2
2
0
1
1
0
4
3
1
0
3
2
3
2
3
1
3
1
3
1
2
1
2
1
2
2
1
2
2
3
2
2
2
4
1
2
3
1
2
3
1
2
2
2
3
1
3
3
2
3
1
3
2
4
3
3
3
3
2
2
3
2
2
2
3
2
4
2
2
2
3
3
1
3
1
2
2
1
2
3
2
1
1
3
2
2
2
2
1
1
2
1
1
1
2
1
2
3
4
2
1
3
2
2
2
3
3
1
3
1
0
2
4
2
2
2
2
3
2
2
2
1
1
1
1
3
2
2
2
2
1
1
2
3
2
3
2
2
4
2
3
3
2
1
2
2
3
2
1
3
3
1
2
3
1
2
2
3
3
2
3
2
2
2
1
3
2
2
2
3
4
1
2
1
3
2
2
3
4
2
3
2
2
3
2
2
2
3
3
1
3
2
3
2
1
3
1
3
2
2
3
1
2
1
2
3
2
3


False